# Goal statement

need to predict a players total points in the tourney as a function of their season points total, their avg points per game, and Tournament Seed

## Data 

below we are trying to ensure we can get:
- the players name, 
- the school they play for
-  conference the school is in
- point per game average
- total points for season

In [14]:
import pandas as pd
from fuzzywuzzy import fuzz
import os

In [103]:


def get_season_year(file):
    return(os.path.basename(file).split('.')[0].split('_')[-1])


def split_player_name(row):

    player = row['Player']
    
    array = player.split(',')
    c = len(array)
    print_it = False
    pn: str = ''
    school: str = ''
    conf: str = ''

    if c == 2:
        pn = array[0]
        a = array[1].split('(')
    elif c == 3:
        pn = f'{array[0]} {array[1].strip()}'
        a = array[2].split('(')
        print_it = True
    else:
        raise KeyError(f'ERROR: Could not determine Name, school and conference {player}')
    
    if len(a) == 2:
        school = a[0]
        conf = a[1]
        
    elif len(a) == 3:
        conf = a[2]
        school = f'{a[0].strip()} {a[1].strip()}'.replace(')', '')
        
    else:
        raise KeyError(f'ERROR: Could not determine school and conference {a}')
    
    conf = conf.replace(')','')
    return pn.strip(), school.strip(), conf.strip()
  

def read_data(file):
    df = pd.read_csv(file, dtype=str, index_col='Rank')
    df['Year'] = get_season_year(file)
    df[['Name','School','Conf']] = df.apply(split_player_name, axis=1, result_type='expand')

    return df


player_stats = read_data('/workspaces/ncaa-player-pool/data/prediction/ncaa_statistics_2022-23.csv')

display(player_stats)






,Player,Cl,Ht,Pos,G,FGM,3FG,FT,PTS,PPG,Year,Name,School,Conf
Rank,,,,,,,,,,,,,,
1,"Antoine Davis, Detroit Mercy (Horizon)",Sr.,6-1,G,33,303,159,165,930,28.2,2022-23,Antoine Davis,Detroit Mercy,Horizon
4,"Darius McGhee, Liberty (ASUN)",Sr.,5-9,G,33,249,153,99,750,22.7,2022-23,Darius McGhee,Liberty,ASUN
7,"Taevion Kinsey, Marshall (Sun Belt)",Sr.,6-5,G,32,271,19,145,706,22.1,2022-23,Taevion Kinsey,Marshall,Sun Belt
15,"Drew Pember, UNC Asheville (Big South)",Sr.,6-10,F,33,192,55,252,691,20.9,2022-23,Drew Pember,UNC Asheville,Big South
8,"Max Abmas, Oral Roberts (Summit League)",Sr.,6-0,G,31,216,111,139,682,22,2022-23,Max Abmas,Oral Roberts,Summit League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,"Jaykwon Walton, Wichita St. (AAC)",Jr.,6-7,G,26,127,36,75,365,14,2022-23,Jaykwon Walton,Wichita St.,AAC
300,"Briggs McClain, Hartford (DI Independent)",Jr.,6-4,G,25,139,39,44,361,14.4,2022-23,Briggs McClain,Hartford,DI Independent
211,"Dawson Garcia, Minnesota (Big Ten)",So.,6-11,F,23,132,25,66,355,15.4,2022-23,Dawson Garcia,Minnesota,Big Ten


Next need work on Matching the above School name with the School seeds data


In [10]:
def get_schools(ranking_data):
    return ranking_data.TEAM.unique()

def match_schools(school: str, schools):
    found_school: bool = False
    print(school)
    if school in schools:
        print(f'FOUND: {school}')
        found_school = True
    else:
        for s in schools:
            if fuzz.ratio(school, s) >= .9:
                print(f'{school} ==> {s}')
    return found_school
        
   

rank_file = '/workspaces/ncaa-player-pool/data/prediction/archive/538 Ratings.csv'

rank_df = pd.read_csv(rank_file, dtype=str, header=0, index_col='TEAM NO')
#print(rank_df)
schools = get_schools(ranking_data=rank_df)
print(schools)
if match_schools(school=school, schools=schools):
    print(school)

['Houston' 'Alabama' 'Kansas' 'Purdue' 'Texas' 'Arizona' 'UCLA'
 'Marquette' 'Gonzaga' 'Baylor' 'Kansas St.' 'Xavier' 'Connecticut'
 'Tennessee' 'Indiana' 'Virginia' 'Duke' 'San Diego St.' "Saint Mary's"
 'Miami FL' 'Creighton' 'Kentucky' 'TCU' 'Iowa St.' 'Texas A&M'
 'Michigan St.' 'Northwestern' 'Missouri' 'Arkansas' 'Memphis' 'Maryland'
 'Iowa' 'Auburn' 'Illinois' 'West Virginia' 'Florida Atlantic' 'Utah St.'
 'USC' 'Penn St.' 'Boise St.' 'Providence' 'Mississippi St.'
 'North Carolina St.' 'Arizona St.' 'Pittsburgh' 'Nevada' 'VCU' 'Drake'
 'College of Charleston' 'Oral Roberts' 'Iona' 'Kent St.' 'Furman'
 'Louisiana Lafayette' 'Grand Canyon' 'UC Santa Barbara' 'Montana St.'
 'Kennesaw St.' 'Vermont' 'Colgate' 'Princeton' 'UNC Asheville'
 'Northern Kentucky' 'Texas A&M Corpus Chris' 'Howard'
 'Southeast Missouri St.' 'Texas Southern' 'Fairleigh Dickinson'
 'Villanova' 'Texas Tech' 'Wisconsin' 'LSU' 'Colorado St.' 'Ohio St.'
 'Murray St.' 'North Carolina' 'Seton Hall' 'Loyola Chicago

In [47]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
X = [[ 1,  2,  3],  # 2 samples, 3 features
     [11, 12, 13]]
y = [0, 1]  # classes of each sample
clf.fit(X, y)

a = clf.predict([[4, 5, 6], [14, 15, 16]])
print(a)


df = pd.DataFrame([[1, 2.12], [3.356, 4.567]])
df
df.map(lambda x: len(str(x)))

[0 1]


,0,1
0,3,4
1,5,5
